# TELESCOPIO JAMES WEBB
## Realizado por: David Guachamín

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tkinter as tk
from tkinter import simpledialog

# Función para calcular la ecuación de la recta que pasa por dos puntos
def ecuacion_recta(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    m = (y2 - y1) / (x2 - x1)
    b = y1 - m * x1
    return m, b

# Función para calcular la ecuación de la recta perpendicular que pasa por un punto
def ecuacion_recta_perpendicular(p1, m_original):
    x0, y0 = p1
    if m_original != 0:
        m_perpendicular = -1 / m_original
    else:
        m_perpendicular = None  # Caso especial: la recta original es horizontal
    if m_perpendicular is not None:
        b_perpendicular = y0 - m_perpendicular * x0
        return m_perpendicular, b_perpendicular
    else:
        return m_perpendicular

# Función para calcular los extremos de una recta perpendicular dada una longitud y un punto
def extremos_recta_perpendicular(punto, m_perpendicular, longitud):
    x0, y0 = punto
    if m_perpendicular is not None:
        d = longitud / 2 / np.sqrt(1 + m_perpendicular**2)
        x1 = x0 - d
        y1 = m_perpendicular * x1 + (y0 - m_perpendicular * x0)
        x2 = x0 + d
        y2 = m_perpendicular * x2 + (y0 - m_perpendicular * x0)
        return (x1, y1), (x2, y2)
    else:
        x1 = x0
        y1 = y0 - longitud / 2
        x2 = x0
        y2 = y0 + longitud / 2
        return (x1, y1), (x2, y2)

# Función para calcular la distancia entre dos puntos
def distancia(p1, p2):
    return np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)

# Función para graficar puntos y rectas
def graficar_puntos_y_rectas(L, B, Dmin, Dmax, Px, Py, F):
    # Convertir F en punto
    punto_foco = (F, 0)

    # Calcular D
    D = Dmin + Dmax

    # Crear la lista de vértices para graficar el cuadrado
    cuadrado_x = [0, 0, B, B, 0]
    cuadrado_y = [0, D, D, 0, 0]

    # Calcular la ecuación de la recta que pasa por P y F
    m_original, b_original = ecuacion_recta((Px, Py), (F, 0))

    # Generar puntos a lo largo de la recta P-F con una separación de 0.5
    distancia_total = np.sqrt((F - Px)**2 + (0 - Py)**2)
    num_puntos = int(distancia_total / 0.5) + 1
    puntos = [(Px + i * 0.5 * (F - Px) / distancia_total, Py + i * 0.5 * (0 - Py) / distancia_total) for i in range(num_puntos)]

    # Calcular la pendiente de la recta perpendicular
    m_perpendicular, _ = ecuacion_recta_perpendicular((Px, Py), m_original)

    # Variables para almacenar la paralela con la mayor suma de longitudes
    max_longitud_suma = 0
    mejor_paralela = None
    mejor_R = None
    mejor_T = None

    # Graficar
    plt.figure(figsize=(12, 8))
    
    # Graficar el cuadrado
    plt.plot(cuadrado_x, cuadrado_y, label="Cuadrado", color="blue")
    
    # Graficar los puntos principales
    plt.scatter(0, 0, color="black", label="A (0,0)")
    plt.scatter(B, 0, color="blue", label=f"B ({B},0)")
    plt.scatter(Px, Py, color="orange", label=f"P ({Px},{Py})")
    plt.scatter(F, 0, color="purple", label=f"F ({F},0)")

    # Graficar la recta P-F con línea entrecortada
    x_vals_PF = np.linspace(Px, F, 500)
    y_vals_PF = m_original * x_vals_PF + b_original
    plt.plot(x_vals_PF, y_vals_PF, 'k--', label="Recta P-F")

    # Iterar y encontrar la paralela con la mayor suma de longitudes
    for i, punto in enumerate(puntos):
        R, T = extremos_recta_perpendicular(punto, m_perpendicular, L)
        distancia_RA = distancia(R, (0, 0))
        distancia_TB = distancia(T, (B, 0))
        
        # Verificar las condiciones de distancia
        if Dmin <= distancia_RA <= Dmin + Dmax and Dmin <= distancia_TB <= Dmin + Dmax:
            suma_longitudes = distancia_RA + distancia_TB
            if suma_longitudes > max_longitud_suma:
                max_longitud_suma = suma_longitudes
                mejor_paralela = (R, T)
                mejor_R = R
                mejor_T = T

    # Graficar la mejor paralela encontrada
    if mejor_paralela:
        R, T = mejor_paralela
        plt.plot([R[0], T[0]], [R[1], T[1]], color="green", label="Mejor Paralela")

        # Graficar extremos
        plt.scatter(*R, color="purple", s=20)
        plt.scatter(*T, color="purple", s=20)
        plt.text(R[0], R[1], f"R ({R[0]:.2f},{R[1]:.2f})", fontsize=8, color="purple", ha='right')
        plt.text(T[0], T[1], f"T ({T[0]:.2f},{T[1]:.2f})", fontsize=8, color="purple", ha='left')

        # Calcular y mostrar las distancias de R a A y T a B
        distancia_RA = distancia(R, (0, 0))
        distancia_TB = distancia(T, (B, 0))
        plt.text(R[0] / 2, R[1] / 2, f"{distancia_RA:.2f}", fontsize=8, color="red", va='center')
        plt.text((B + T[0]) / 2, (T[1]) / 2, f"{distancia_TB:.2f}", fontsize=8, color="brown", va='center')

        # Anotaciones para X1 y X2 en las rectas R-A y T-B
        X1 = distancia_RA - Dmin
        X2 = distancia_TB - Dmin
        plt.text(R[0] / 2, R[1] / 2 + 0.5, f"X1 = {X1:.2f}", fontsize=10, color="blue")
        plt.text((B + T[0]) / 2, (T[1]) / 2 + 0.5, f"X2 = {X2:.2f}", fontsize=10, color="blue")

        # Unir extremos izquierdos al punto A (línea sólida)
        plt.plot([R[0], 0], [R[1], 0], color="orange", label="R-A")

        # Unir extremos derechos al punto B (línea sólida)
        plt.plot([T[0], B], [T[1], 0], color="brown", label="T-B")

    # Agregar etiquetas y leyendas
    plt.title("TELESCOPIO JAMES WEBB")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.grid(color='gray', linestyle='--', linewidth=0.5)
    plt.legend()
    plt.show()

# Función para obtener los parámetros desde la interfaz gráfica
def obtener_parametros():
    root = tk.Tk()
    root.withdraw()  # Oculta la ventana principal

    L = simpledialog.askfloat("Input", "Ingrese L (ej. 8):")
    B = simpledialog.askfloat("Input", "Ingrese B (ej. 11):")
    Dmin = simpledialog.askfloat("Input", "Ingrese Dmin (ej. 7):")
    Dmax = simpledialog.askfloat("Input", "Ingrese Dmax (ej. 9):")
    Px = simpledialog.askfloat("Input", "Ingrese Px (ej. 18):")
    Py = simpledialog.askfloat("Input", "Ingrese Py (ej. 22):")
    F = simpledialog.askfloat("Input", "Ingrese F (ej. 3):")

    if None not in (L, B, Dmin, Dmax, Px, Py, F):
        graficar_puntos_y_rectas(L, B, Dmin, Dmax, Px, Py, F)

# Ejecutar la función para obtener los parámetros y graficar
if __name__ == "__main__":
    obtener_parametros()
